In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from classifier import Data

dataset = Data(
    data_path = "../data/imdb.train.csv",
    polarities = {
      "negative": 0,
      "positive": 1
    },
    data_label='review',
    target_label='sentiment'
)

In [4]:
from classifier._neural import Encoder

enc = Encoder({
    'model': 'bert-base-uncased',
    'layers': [11]
})

In [5]:
enc.df_encode(dataset.data, col=dataset.data_label, label=dataset.data_path)

In [6]:
import numpy as np
from sklearn.manifold import TSNE

def manifold_reduction(data: np.ndarray, dim: int = 3) -> np.ndarray:
    return TSNE(
            n_components=dim,
            learning_rate='auto',
            init='random'
        ).fit_transform(data)

In [7]:
import torch

prepared_data: np.ndarray = torch.stack(dataset.data[enc.col_name].tolist()).numpy()

In [8]:
reduced_data = manifold_reduction(prepared_data)
reduced_data.shape

(20170, 3)

In [9]:
dataset.data['reduced_embeds'] = list(reduced_data)

In [10]:
for _, group in dataset.data.groupby(dataset.target_label):
    print(group)

                                                  review sentiment  \
1      This is a strange sex comedy because there`s v...  negative   
3      The premise is ridiculous, the characters unbe...  negative   
5      This was just a terrible movie. It hurt me to ...  negative   
6      I cannot say that Aag is the worst Bollywood f...  negative   
13     I firmly believe that the best Oscar ceremony ...  negative   
...                                                  ...       ...   
20164  Former brat pack actor and all round pretty bo...  negative   
20165  The plot of 'Edison' was decent, but one actor...  negative   
20166  Amateurism best describes the film adaptation ...  negative   
20167  After having seen the movie the first question...  negative   
20169  Sholay: Considered to be one of the greatest f...  negative   

                                              prediction  \
1      [tensor(-0.2426), tensor(-0.2292), tensor(0.58...   
3      [tensor(0.1673), tensor(-0.1038)